#Simulation functions  
OJM  
Similarly to data analysis functions, separate out here.

In [1]:
import numpy as np
import scipy as sp
from scipy import stats
import scipy.interpolate as interpolate
import scipy.special as special
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import errno
%matplotlib inline
from clawpack import pyclaw
from clawpack import riemann
#%load_ext autoreload #are these needed in notebook??
#%autoreload 2 #are these needed in notebook??

In [1]:
%run data_analysis_functions.ipynb

####proliferation and velocity

In [3]:
def proliferation_profile(p=np.array([1.0,2.0,0.001,0.001]),x_max=1.):
    """
    simple piecewise constant proliferation profile function
    k(x)  = p_i, x_0<x<x_i for x in [0,1]
    -
    notes:
    p_i includes the very right-most point - only used at edge of domain. Hence -1.0 in indexing.
    evaluate at multiple points, if necessary, using list comprehensions, e.g. [k(x_i) for x_i in x]
    -
    todos:
    constrain so non-negative near base?
    """
    
    k_prolif= lambda x: p[np.floor((x/x_max)*(p.size-1.0))]
    
    #p_grid= np.linspace(0.,1.0,p.size+1)
    #k_prolif=lambda xi: [p[i] for i in range(0,p_grid.size-1) if (xi >= x_max*p_grid[i]) & (xi <= x_max*p_grid[i+1])][0]
    #p=np.array([1.0,2.0,0.0,0.5])
    #grid= np.linspace(0.,1.0,p.size+1)
    #print p
    #print grid
    #k_prolif= lambda xi: np.piecewise(xi,[(xi >= grid[i]) & (xi <= grid[i+1]) for i in range(len(grid)-1)],p)
    #grid= np.linspace(0.,1.0,p.size+1)
    
    #vector evaluation?
    #k_prolif= lambda x_i: np.piecewise(x_i,[(x_i >= x_max*p_grid[i]) & (x_i <= x_max*p_grid[i+1]) for i in range(len(p_grid)-1)],p)
    
    return k_prolif

In [4]:
def velocity_from_integrated_proliferation(k_prolif,x_lower=0.0):
    """
    velocity as defined by integral of proliferation function
    -
    notes:
    could be given directly then differentiated? what does this do? any bias?
    in many cases could be integrated analytically
    is this numerical scheme any good?
    currently makes no use of previous integrations - could store results/give recursively?
    argument for using e.g. splines/gps? - can integrate once
    """
    #could be much more efficient! can express analytically duh!
    return lambda x_i: sp.integrate.quad(k_prolif,x_lower,x_i)[0]

####pyclaw functions

In [3]:
def setup(nx=100, kernel_language='Python', use_petsc=False, solver_type='classic', weno_order=5, 
    time_integrator='SSP104', outdir_claw='./_output',initial_profile_f = lambda x: x, 
    velocity_f= lambda x: x,norm_out_times= np.linspace(0.0,1.0,10),domain_length=100.):

    """
    based on example ??? from pyclaw. updated to use color equation riemann solver.
    sets up a pyclaw controller.
    -
    options:
    normalised time?
    un-normalised space!
    -
    todos:
    write own riemann solver
    kernal language?
    re-check boundary conditions?
    ?
    """
    
    #--solver options--
    if use_petsc:
        import clawpack.petclaw as pyclaw
    else:
        from clawpack import pyclaw

    #NEEDS TO BE SET TO PYTHON! Go over.
    if kernel_language == 'Fortran':
        #riemann_solver = riemann.advection_1D #OLD SOLVER
        riemann_solver = riemann.vc_advection_1D
    elif kernel_language == 'Python':
        #riemann_solver = riemann.advection_1D_py.advection_1D #OLD SOLVER
        riemann_solver = riemann.vc_advection_1D_py.vc_advection_1D

    if solver_type=='classic':
        solver = pyclaw.ClawSolver1D(riemann_solver)
    elif solver_type=='sharpclaw':
        solver = pyclaw.SharpClawSolver1D(riemann_solver)
        solver.weno_order = weno_order
        solver.time_integrator = time_integrator
    else: raise Exception('Unrecognized value of solver_type.')

    solver.kernel_language = kernel_language #NEEDS TO BE SET TO PYTHON
    #--
    
    #--boundary conditions--
    #label
    solver.bc_lower[0] = pyclaw.BC.extrap
    solver.bc_upper[0] = pyclaw.BC.extrap
    #velocity
    solver.aux_bc_lower[0] = pyclaw.BC.wall
    solver.aux_bc_upper[0] = pyclaw.BC.extrap

    #--domain and state--
    x = pyclaw.Dimension('x',0.0,domain_length,nx)
    domain = pyclaw.Domain(x)
    num_aux= 1 #velocity
    state = pyclaw.State(domain,solver.num_eqn,num_aux)

    #--initial data & time-independent aux--
    #TODO FUNCTION EVALUATIONS AND LIST COMP.
    x_centres = state.grid.x.centers #CAREFUL OF CELL CENTRES?? TODO
    state.q[0,:] = [initial_profile_f(xc) for xc in x_centres]
    state.aux[0,:]= [velocity_f(xc) for xc in x_centres]
    
    #--controller and solver--
    claw = pyclaw.Controller()
    claw.keep_copy = True
    claw.solution = pyclaw.Solution(state,domain)
    claw.solver = solver

    if outdir_claw is not None:
        claw.outdir = outdir_claw
    else:
        claw.output_format = None
    
    claw.t0 = 0.0 #normed time?
    claw.tfinal =1.0 #normed time?
    claw.output_style = 2 #required for out_times? see help.
    claw.out_times= norm_out_times
    
    #import logging
    #solver.logger.setLevel(logging.CRITICAL)
    claw.verbosity = 0
    
    #import logging
    #logger = logging.getLogger('claw')
    #logger.setLevel(logging.CRITICAL)
    #claw.setplot = setplot #required?

    return claw

def setplot(plotdata):
    """ 
    Plot solution using VisClaw.
    """ 
    plotdata.clearfigures()  # clear any old figures,axes,items data

    plotfigure = plotdata.new_plotfigure(name='q', figno=1)

    # Set up for axes in this figure:
    plotaxes = plotfigure.new_plotaxes()
    plotaxes.ylimits = [-.2,1.0]
    plotaxes.title = 'q'

    # Set up for item on these axes:
    plotitem = plotaxes.new_plotitem(plot_type='1d_plot')
    plotitem.plot_var = 0
    plotitem.plotstyle = '-'
    plotitem.color = 'b'
    plotitem.kwargs = {'linewidth':2,'markersize':5}

    return plotdata

####Single sim and multiple/optimisation sims

In [2]:
def construct_and_run_sim(data_dir='../data-working/TXT_BrdU/',sample_type='BrdU',
                          actual_out_times= np.array([60.,120.,360.,600.,1080.,1920.]),precision_time=4,
                          p=np.array([1.0,2.0,0.01,0.01,0.01]),x_min=0.,x_max=100.,nx=100):
    """
    takes a directory, set of times and 
    finds start file corresponding to first time in set
    gets initial profile fit
    uses as IC
    solves
    returns simulation results at desired times in np arrays
    """
    #get initial condition
    files_in_dir= os.listdir(data_dir)
    files_in_dir.sort() #assumes files have same name format!!

    start= actual_out_times[0]
    time_format= '%0'+('%1d' % precision_time)+'d'
    file_ic= get_data_file(data_dir,time_format%start)
    
    density_results= process_and_fit_label_data_file(data_dir=data_dir,file_to_fit=file_ic,sample_type=sample_type,x_max=x_max,do_plot=False)
    initial_profile_f= density_results[-1]
    
    velocity_f= velocity_from_integrated_proliferation(proliferation_profile(p=p,x_max=x_max))
    
    #convert between experimental times and simulation times 
    norm_out_times= (actual_out_times-min(actual_out_times))/(max(actual_out_times)-min(actual_out_times))

    #set up and run simulation 
    controller= setup(nx=nx,initial_profile_f=initial_profile_f,velocity_f=velocity_f,norm_out_times=norm_out_times)
    #controller.verbosity= 0
    controller.run()

    #extract (all) simulation results
    output_shape= [np.size(controller.frames[0].state.q[0,:],axis=0),np.size(controller.frames,axis=0)]
    #print output_shape
    labels= np.zeros(output_shape)
    x_centres= np.zeros(output_shape)
    velocity= np.zeros(output_shape)
    for i in range(0,np.size(controller.out_times,axis=0)):
        labels[:,i]= controller.frames[i].state.q[0,:]
        #don't actually vary with time!
        x_centres[:,i]= controller.frames[0].state.grid.c_centers[0]
        velocity[:,i]= controller.frames[0].state.aux[0,:]

    return labels, velocity, x_centres

def opt_sim(data_dir='../data-working/TXT_BrdU/',sample_type='BrdU',
                          actual_out_times= np.array([60.,120.,360.,600.,1080.,1920.]),times_to_fit_i=[5],precision_time=4,
                          p0=np.array([1.0,2.0,0.01,0.01,0.01]),p_var_i=[],reg_param= 0.1,penalty_order=1,x_min=0.,x_max=100.,k=3,s=15,nx=100):
    """
    optimisation solution to inverse problem
    -
    notes
    a key difficulty is choosing proper comparison grid
    also, systematic regularisation - dependence on number of parameters etc? determining reg. parameter
    -
    structure
    initial condition
    data for fitting
    model definition
    residuals function
    
    
    """
    #---
    #get initial condition
    files_in_dir= os.listdir(data_dir)
    files_in_dir.sort() #assumes files have same name format!!

    start= actual_out_times[0]
    time_format= '%0'+('%1d' % precision_time)+'d'
    file_ic= get_data_file(data_dir,time_format%start)
    
    density_results= process_and_fit_label_data_file(data_dir=data_dir,file_to_fit=file_ic,
                                                     sample_type=sample_type,x_max=x_max,
                                                     do_plot=False)
    initial_profile_f= density_results[-1]
    
    #---
    #data for comparison. NOTE - get 
    #TODO - use times_to_fit_i?
    x_data_to_fit= np.tile(np.arange(x_min,x_max),(actual_out_times.size-1,1))
    x_data_to_fit= np.transpose(x_data_to_fit)
    label_data_at_x_data= np.zeros(x_data_to_fit.shape)

    for i in range(0,actual_out_times.size-1):
        current_time= actual_out_times[i+1]
        file_current= get_data_file(data_dir,time_format%current_time)
        print file_current
        data_result= process_and_fit_label_data_file(data_dir=data_dir,file_to_fit=file_current,sample_type=sample_type,k=k,s=s,x_max=100,do_plot=False)
        #careful here - data grid concept needs to be tidied up.
        label_data_at_x_data[:,i]= np.append(data_result[0],np.zeros(x_max-data_result[0].size))
    
    #convert between experimental times and simulation times 
    norm_out_times= (actual_out_times-min(actual_out_times))/(max(actual_out_times)-min(actual_out_times))
    
    #---
    #function for one sim.
    def model(p_var):
        """
        simulation model
        -
        notes:
        output formats for each quantity are -
        [column of results at time 1|column of results at time 2|...etc...]
        uses arguments from outer function - bad practice?
        bit of a hack with 'global' vs. local arguments.
        """
        #HACK!
        #try_count= 0
        #while try_count<5:
        #    try:
                #code with possible error
                #set up and run simulation 
        #        velocity_f= velocity_from_integrated_proliferation(proliferation_profile(p=p,x_max=x_max))
        #        controller= setup(nx=nx,initial_profile_f=initial_profile_f,velocity_f=velocity_f,norm_out_times=norm_out_times)
        #        #controller.verbosity= 0
        #        controller.run()
        #    except:
        #        print 'adding noise/making positive to try avoid numerical difficulties'
        #        print p
                #print p_current
                #p= p+np.random.normal(0,0.1,p.size)
        #        p= np.abs(p+np.random.normal(0,0.1,p.size))
        #        try_count= try_count+1
        #        continue
        #    else:
        #         #the rest of the code
        #         break
        p= p0
        p[p_var_i]= p_var[p_var_i]
        #print p[p_fixed_i]== p0[p_fixed_i]
        #print 'here'
        velocity_f= velocity_from_integrated_proliferation(proliferation_profile(p=p,x_max=x_max))
        controller= setup(nx=nx,initial_profile_f=initial_profile_f,velocity_f=velocity_f,norm_out_times=norm_out_times)
        #controller.verbosity= 0
        controller.run()

        #extract (all) simulation results
        output_shape= [np.size(controller.frames[0].state.q[0,:],axis=0),np.size(controller.frames,axis=0)]
        #print output_shape
        labels= np.zeros(output_shape)
        x_centres= np.zeros(output_shape)
        velocity= np.zeros(output_shape)
        for i in range(0,np.size(controller.out_times,axis=0)):
            labels[:,i]= controller.frames[i].state.q[0,:]
            #don't actually vary with time!
            x_centres[:,i]= controller.frames[0].state.grid.c_centers[0]
            velocity[:,i]= controller.frames[0].state.aux[0,:]

        return labels, velocity, x_centres

    #---
    #residuals function
    def residuals(p_var_current,flatten_residual=True):#,p0=np.array([1.0,2.0,0.01,0.01,0.01]))#,p_fixed_i=[]):#,x_data_to_fit=x_data_to_fit,label_data_to_fit=label_data_to_fit,times_to_fit=[]):
        """
        residuals between model solutions and data
        -
        notes
        data and model results are matrices/arrays!
        in 'column vector' storage format?
        -
        Plan
        general outline
        -at a given time
        [vectorised]
        --at a set of data comparison x points
        ---get data values
        ---get solution values (via interp.)
        ---compute residual and store as column vector in residual matrix
        -(in another function) square values and sum to get a single scalar
        approach
        -test cell to consider a vector of data values and test sim and calc residual
        """
        #get solutions at all times > t0. #use e.g. structured arrays??
        results= model(p_var=p_var_current)
        labels_model= results[0][:,1:]
        x_centres_model= results[2][:,1:]

        #data grid. TODO - do better. Note: don't include initial condition so one index smaller.
        #use e.g. structured arrays?? For now - collect all but only compare subset. Inefficient.
        label_model_at_x_data_current= np.zeros(x_data_to_fit.shape[0])
        residual_at_x_data= np.zeros(x_data_to_fit.shape)
        #times_to_fit_i
        for i in np.subtract(times_to_fit_i,1):
            #TODO - assert i>0
            
            #current_time= actual_out_times[i+1]
            #file_current= get_data_file(data_dir,time_format%current_time)
            #print file_current
            #data_result= process_and_fit_label_data_file(data_dir=data_dir,file_to_fit=file_current,sample_type=sample_type,k=k,s=s,x_max=100,do_plot=False)
            #careful here - data grid concept needs to be tidied up.
            #label_data_at_x_data= np.append(data_result[0],np.zeros(x_max-data_result[0].size))
            label_model_at_x_data_current= np.interp(x_data_to_fit[:,i],x_centres_model[:,i],labels_model[:,i])
            residual_at_x_data[:,i]= label_data_at_x_data[:,i]-label_model_at_x_data_current
            #temp
            #plt.plot(residual_at_x_data[:,i])

        #temp
        #plt.show()
        if flatten_residual:
            return np.ravel(residual_at_x_data) #ravel flattens into single vector
        else:
            return residual_at_x_data
    #---
    #Optimisation
    #currently only one step.
    import scipy.optimize as sciopt
    flatten_residual= True
    
    #plsq= sciopt.leastsq(residuals,p0,args=(flatten_residual))
    #pest= plsq[0]
    #bounds= [(-10,10) for i in range(p.size)]
    #max_iter= 10
    #max_fev= 20
    #note - use centered parameters in penalty. CAREFUL OF FIXED PARAMETERS!!!
    p_var0= p0[p_var_i]
    sum_square_penalised= lambda p_var: (1/np.float(len(times_to_fit_i)))*np.linalg.norm(residuals(p_var_current=p_var))\
        +reg_param*(np.linalg.norm(p_var-np.mean(p_var),ord=penalty_order))
    #out= sciopt.minimize(sum_sq,p0,bounds=bounds,method=),method='Nelder-Mead'
    #out= sciopt.minimize(sum_square_penalised,p0,method='Nelder-Mead',options=dict({'maxiter':10,'maxfev':20}))
    out= sciopt.minimize(sum_square_penalised,p_var0,method='Nelder-Mead')
    #pest= out.x
    return out

### stochastic model